In [1]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

import sys
sys.path.append('../lib/')
from financelib import FinLoad
from financelib import FinCalc
from financelib import FinFetch
from plotlib import FinPlot
from financelib import FinInvestmentsGet

from pathlib import Path

In [2]:
YEAR = 2025
data_path_o = Path("../../data")

In [3]:
init_holdings = FinLoad.load_init_holdings(data_path_o, YEAR)
df_year_cashflow = FinLoad.load_cashflow(data_path_o, YEAR)
df_year_investments = FinLoad.load_investments(data_path_o, YEAR)

In [4]:
df_m_cashflow = FinCalc.calc_monthly_cashflow(df_year_cashflow, init_holdings, YEAR)

In [5]:
df_m_cashflow

,incomes,liabilities,savings,saving_rate,investments,liquidity
2024-12-31,-,-,"14,711.19",-,0.00,"14,711.19"
2025-01-31,"2,231.00",-626.02,"1,604.98",0.72,0.00,"16,316.17"
2025-02-28,"1,811.00",-632.58,"1,178.42",0.65,-139.42,"17,355.17"
2025-03-31,"1,746.87",-633.45,"1,113.42",0.64,-263.75,"18,204.84"
2025-04-30,"1,652.49",-310.46,"1,342.03",0.81,-245.92,"19,300.95"
2025-05-31,"1,645.03",-561.96,"1,083.07",0.66,-253.95,"20,130.07"


In [6]:
fig_cashflow = FinPlot.plot_cashflow(df_m_cashflow)
fig_cashflow.show()

### Current balance

In [7]:
current_holdings = FinCalc.calc_current_balance(df_year_cashflow, init_holdings)
tot = 0
for key in current_holdings.keys():
    tot += current_holdings[key]
print(tot)

21813.3


In [8]:
holdings_last_month = FinCalc.calc_balance_last_day_previous_month(df_year_cashflow, init_holdings, YEAR)
tot = 0
for key in holdings_last_month.keys():
    tot += holdings_last_month[key]
print(tot)

20330.07


In [9]:
df_expenses = FinCalc.calc_expenses(df_year_cashflow[ df_year_cashflow.index.month == 5 ])
df_expenses["Qty"] = df_expenses.Qty.abs() # sunburst does not understand negative values
fig = FinPlot.plot_expenses_donut(df_expenses)
fig.show()

### Investments 2025

In [10]:
init_holdings = FinLoad.load_init_holdings(data_path_o, YEAR)
df_year_investments = FinLoad.load_investments(data_path_o, YEAR)

In [11]:
df_init_investments = FinInvestmentsGet.get_init_holdings_to_df(init_holdings, YEAR)
df_year_investments = pd.concat([df_init_investments, df_year_investments])


In [12]:
holdings_monthlyized = FinInvestmentsGet.get_holdings_monthlyized(df_year_investments, YEAR)


In [13]:
assets_monthlyized = FinInvestmentsGet.get_assets_monthlyized(holdings_monthlyized, data_path_o, YEAR)
assets = FinInvestmentsGet.get_assets_global(assets_monthlyized, holdings_monthlyized)

SOL
New data must be downloaded
Updated asset data saved in local to ../../data/2025/investments/exchange/SOL-EUR.csv
ETH
New data must be downloaded
Updated asset data saved in local to ../../data/2025/investments/exchange/ETH-EUR.csv
USDT
New data must be downloaded
Updated asset data saved in local to ../../data/2025/investments/exchange/USDT-EUR.csv
IE00BK5BQT80
New data must be downloaded
Updated asset data saved in local to ../../data/2025/investments/exchange/IE00BK5BQT80-EUR.csv
JE00B1VS3770
New data must be downloaded
Updated asset data saved in local to ../../data/2025/investments/exchange/JE00B1VS3770-EUR.csv


In [14]:
df_year_holdings = FinInvestmentsGet.get_total_holdings(assets)

In [15]:
df_year_holdings.T

,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31
SOL,892.26,"1,117.39",665.78,579.09,646.47,692.34
ETH,38.43,37.22,0.00,0.00,0.00,0.00
USDT,103.68,103.68,103.68,99.36,95.04,95.04
IE00BK5BQT80,0.00,0.00,136.61,379.74,607.90,902.23
JE00B1VS3770,0.00,0.00,0.00,0.00,0.00,0.00
Total,"1,034.37","1,258.29",906.08,"1,058.19","1,349.42","1,689.61"


### Calculate current month
Which is adding a column with current day and all my assets evaluated at the present moment in real time.

#### Current month cashflow
It's different from monthly view because this does not have the last day of december.

In [16]:
row_today_cashflow = FinCalc.calc_curr_month_cashflow(df_year_cashflow, df_m_cashflow)

#### Current month investments
It's different from monthly view because this does not have the last day of december.

In [17]:
current_holdings = FinInvestmentsGet.get_current_holdings(df_year_investments, holdings_monthlyized)
assets_current_day = FinInvestmentsGet.get_current_assets_price(current_holdings, assets_monthlyized)
assets_global_current_day = FinInvestmentsGet.get_current_assets_holdings(assets_current_day, current_holdings)
df_today_holdings = FinInvestmentsGet.get_total_holdings(assets_global_current_day)

SOL
ETH
USDT
IE00BK5BQT80
JE00B1VS3770
Getting SOL today EUR price...
Getting ETH today EUR price...
Getting USDT today EUR price...
Getting IE00BK5BQT80 today EUR price...
Getting JE00B1VS3770 today EUR price...


### Current Day NW

In [18]:
nw_global = FinCalc.calc_global_nw(row_today_cashflow, df_today_holdings, df_m_cashflow, df_year_holdings)

In [21]:
nw_global.T

,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31,2025-06-04
liquidity,"14,711.19","16,316.17","17,355.17","18,204.84","19,300.95","20,130.07","21,613.30"
investments,"1,034.37","1,258.29",906.08,"1,058.19","1,349.42","1,689.61","1,821.46"
networth,"15,745.56","17,574.46","18,261.25","19,263.03","20,650.37","21,819.68","23,434.76"
nwch,NaN,"1,828.90",686.79,"1,001.79","1,387.33","1,169.31","1,615.08"
ch%,NaN,0.10,0.04,0.05,0.07,0.05,0.07


In [19]:
nw_global.iloc[-1].T

liquidity     21,613.30
investments    1,821.46
networth      23,434.76
nwch           1,615.08
ch%                0.07
Name: 2025-06-04 00:00:00, dtype: float64

In [20]:
df_today_holdings

,SOL,ETH,USDT,IE00BK5BQT80,JE00B1VS3770,Total
2025-06-04,689.62,0.00,95.04,"1,036.80",0.00,"1,821.46"
